In [1]:
!pip install pyspark


In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName("NetflixSQL").getOrCreate()


In [4]:
from google.colab import files
uploaded = files.upload()


Saving netflix_titles.csv to netflix_titles (1).csv


In [5]:
df = spark.read.csv("/content/netflix_titles (1).csv", header=True, inferSchema=True)


In [6]:
# Show schema & sample
df.printSchema()
df.show(5)

root
 |-- show_id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- title: string (nullable = true)
 |-- director: string (nullable = true)
 |-- cast: string (nullable = true)
 |-- country: string (nullable = true)
 |-- date_added: string (nullable = true)
 |-- release_year: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- listed_in: string (nullable = true)
 |-- description: string (nullable = true)

+-------+-------+--------------------+---------------+--------------------+-------------+------------------+------------+------+---------+--------------------+--------------------+
|show_id|   type|               title|       director|                cast|      country|        date_added|release_year|rating| duration|           listed_in|         description|
+-------+-------+--------------------+---------------+--------------------+-------------+------------------+------------+------+---------+-------------------

In [7]:
df.createOrReplaceTempView("netflix")


SQL queries

In [9]:
#1. Total number of Movies vs TV Shows
spark.sql("""
    SELECT type, COUNT(*) as total
    FROM netflix
    GROUP BY type
""").show()

+-------------+-----+
|         type|total|
+-------------+-----+
|         NULL|    1|
|      TV Show| 2676|
|        Movie| 6131|
|William Wyler|    1|
+-------------+-----+



In [10]:
#2. Top 10 countries with most content
spark.sql("""
    SELECT country, COUNT(*) as total_content
    FROM netflix
    WHERE country IS NOT NULL
    GROUP BY country
    ORDER BY total_content DESC
    LIMIT 10
""").show()


+--------------+-------------+
|       country|total_content|
+--------------+-------------+
| United States|         2805|
|         India|          972|
|United Kingdom|          419|
|         Japan|          245|
|   South Korea|          199|
|        Canada|          181|
|         Spain|          145|
|        France|          123|
|        Mexico|          110|
|         Egypt|          106|
+--------------+-------------+



In [11]:
#3. Number of titles released each year
spark.sql("""
    SELECT release_year, COUNT(*) as total_titles
    FROM netflix
    GROUP BY release_year
    ORDER BY release_year DESC
""").show(10)

+-----------------+------------+
|     release_year|total_titles|
+-----------------+------------+
|    United States|           1|
|    June 12, 2021|           1|
| January 15, 2021|           1|
| January 13, 2021|           1|
|December 15, 2020|           1|
|  August 13, 2020|           1|
|           40 min|           1|
|             2021|         589|
|             2020|         952|
|             2019|        1026|
+-----------------+------------+
only showing top 10 rows



In [12]:
#4. Most common ratings
spark.sql("""
    SELECT rating, COUNT(*) as total
    FROM netflix
    GROUP BY rating
    ORDER BY total DESC
""").show()


+-----------------+-----+
|           rating|total|
+-----------------+-----+
|            TV-MA| 3195|
|            TV-14| 2158|
|            TV-PG|  862|
|                R|  796|
|            PG-13|  489|
|            TV-Y7|  334|
|             TV-Y|  307|
|               PG|  286|
|             TV-G|  220|
|               NR|   80|
|                G|   41|
|             NULL|    6|
|         TV-Y7-FV|    6|
|               UR|    3|
|            NC-17|    3|
|             2021|    2|
| November 1, 2020|    1|
| Shavidee Trotter|    1|
|    Adriane Lenox|    1|
|    Maury Chaykin|    1|
+-----------------+-----+
only showing top 20 rows



In [13]:
#5. Filter – All Indian Movies
spark.sql("""
    SELECT title, release_year, rating
    FROM netflix
    WHERE country = 'India' AND type = 'Movie'
    ORDER BY release_year DESC
""").show(10)

+--------------------+------------+------+
|               title|release_year|rating|
+--------------------+------------+------+
|  Sardar Ka Grandson|        2021| TV-14|
|Searching For Sheela|        2021| TV-14|
|               Ahaan|        2021| TV-MA|
|             Sarbath|        2021| TV-PG|
|        Cinema Bandi|        2021| TV-MA|
|          Thimmarusu|        2021| TV-14|
|           Milestone|        2021| TV-14|
|            99 Songs|        2021| TV-14|
|     Haseen Dillruba|        2021| TV-MA|
|             Nayattu|        2021| TV-MA|
+--------------------+------------+------+
only showing top 10 rows



In [14]:
#6. Top 5 Directors with Most Content on Netflix
spark.sql("""
    SELECT director, COUNT(*) as total_titles
    FROM netflix
    WHERE director IS NOT NULL
    GROUP BY director
    ORDER BY total_titles DESC
    LIMIT 5
""").show()

+--------------------+------------+
|            director|total_titles|
+--------------------+------------+
|       Rajiv Chilaka|          19|
|Raúl Campos, Jan ...|          18|
|        Marcus Raboy|          16|
|         Suhas Kadav|          16|
|           Jay Karas|          14|
+--------------------+------------+



In [15]:
#7. Longest Description (title with most words in description)
from pyspark.sql.functions import size, split

df2 = df.withColumn("desc_length", size(split(df["description"], " ")))
df2.createOrReplaceTempView("netflix_desc")

spark.sql("""
    SELECT title, desc_length
    FROM netflix_desc
    ORDER BY desc_length DESC
    LIMIT 5
""").show()

+--------------------+-----------+
|               title|desc_length|
+--------------------+-----------+
|               Billu|         48|
|            Snow Day|         44|
|Shootout at Lokha...|         44|
| Balto 2: Wolf Quest|         44|
|Opium and the Kun...|         42|
+--------------------+-----------+



In [16]:
#8. Trend of TV Shows vs Movies Over Time
spark.sql("""
    SELECT release_year, type, COUNT(*) as total
    FROM netflix
    GROUP BY release_year, type
    ORDER BY release_year DESC, total DESC
""").show(15)


+-----------------+-------------+-----+
|     release_year|         type|total|
+-----------------+-------------+-----+
|    United States|        Movie|    1|
|    June 12, 2021|        Movie|    1|
| January 15, 2021|        Movie|    1|
| January 13, 2021|        Movie|    1|
|December 15, 2020|      TV Show|    1|
|  August 13, 2020|        Movie|    1|
|           40 min|William Wyler|    1|
|             2021|      TV Show|  314|
|             2021|        Movie|  275|
|             2020|        Movie|  516|
|             2020|      TV Show|  436|
|             2019|        Movie|  630|
|             2019|      TV Show|  396|
|             2018|        Movie|  765|
|             2018|      TV Show|  380|
+-----------------+-------------+-----+
only showing top 15 rows

